In [ ]:
import matplotlib
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
%matplotlib inline
import numpy as np
import pandas
import json
from PIL import Image, ImageDraw
import cPickle as pkl
from matplotlib.colors import LogNorm
from tqdm import tqdm_notebook as tqdm
from  scipy.spatial import distance

In [ ]:
cls_id = pkl.load(file('../common/class_id_map.pkl', 'rb'))

In [ ]:
val_img_strokes = {}

In [ ]:
from infer import InferUtil

In [ ]:
iu = InferUtil('./ckpt/model-99001')

In [ ]:
for cls in tqdm(cls_id):
    _tmp = pandas.read_csv('../../data_all/%s.csv'%cls)
    n = len(_tmp)
    
    tmp_list = []
    img_list = []
    for val_idx in range(n-1000, n):
        strokes = json.loads(_tmp.iloc[val_idx,1])

        image = Image.new("P", (128,128), color=255)
        image_draw = ImageDraw.Draw(image)
        
        save_strokes = []
        
        for stroke in strokes:
            ss = np.array(stroke)/2
            for i in range(len(ss[0])-1):
                image_draw.line([ss[0][i], 
                                 ss[1][i],
                                 ss[0][i+1], 
                                 ss[1][i+1]],
                                fill=0, width=3)
            save_strokes.append(ss)
        tmp_list.append(save_strokes)
        img_list.append(np.array(image, dtype=np.uint8))
    
    feats = iu.ext(img_list)    
    
    val_img_strokes[cls] = list(zip(feats, tmp_list))

In [ ]:
pkl.dump(val_img_strokes, file('val_image_strokes.pkl','wb'), 2)